In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from xgboost import XGBClassifier
import catboost as cb
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer

In [3]:
train_meta_or = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv',low_memory=False)
test_meta_or = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv',low_memory=False)

train_meta = train_meta_or.drop('isic_id', axis=1)
test_meta = test_meta_or.drop('isic_id', axis=1)

y_train_df = train_meta['target']

diff_train_test = set(train_meta.columns) - set(test_meta.columns)

x_train_df = train_meta.drop(columns=diff_train_test)

x_test_df = test_meta 

In [4]:
mean = x_train_df["age_approx"].mean()

num_imputer = SimpleImputer(strategy='constant', fill_value=mean)

x_train_df["age_approx"] = num_imputer.fit_transform(x_train_df[["age_approx"]])
x_test_df["age_approx"] = num_imputer.transform(x_test_df[["age_approx"]])

x_train_df = x_train_df.fillna("miss")
x_test_df = x_test_df.fillna("miss")

In [5]:
# x_train_df.info()

In [6]:
len(x_test_df.columns)

43

In [7]:
def calc_auc(solution: pd.DataFrame, submission: pd.DataFrame, min_tpr: float=0.80):
    '''
    Function to calculate tpr from auc
    solution : target
    submission : predict 
    '''
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [8]:
def train_model(main_model,X,y,X_test,n_splits):
    
    skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=42)
    train_scores = []
    val_scores = []
    test_predictions = [] 
    models = []
    
    for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
        
        x_train, x_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        
        model = main_model
        
        model.fit(x_train,y_train)
        
        y_train_pred_proba = model.predict_proba(x_train)[:, 1]
        y_train_pred_df = pd.DataFrame(y_train_pred_proba, columns=['pred'])
        train_pauc = calc_auc(y_train,y_train_pred_df,min_tpr=0.8)
        train_scores.append(train_pauc)
                                 
        y_val_pred_proba = model.predict_proba(x_val)[:, 1]
        y_val_pred_df = pd.DataFrame(y_val_pred_proba, columns=['pred'])
        val_pauc = calc_auc(y_val,y_val_pred_df,min_tpr=0.8)
        val_scores.append(val_pauc)
                                       
        y_test_pred_proba = model.predict_proba(X_test)[:, 1]
        test_predictions.append(y_test_pred_proba)
        
        models.append(model)
        
        print(f"Fold {fold}: Train pAUC = {train_pauc:.4f}, Validation pAUC = {val_pauc:.4f}")
                                       
    mean_train_pauc = np.mean(train_scores)
    mean_val_pauc = np.mean(val_scores)         
    
    print(f"\nMean Train pAUC: {mean_train_pauc:.4f}")
    print(f"Mean Validation pAUC: {mean_val_pauc:.4f}")
                                 
    return model,test_predictions,models

    

In [9]:
cat_cols = ["sex", "tbp_tile_type", "tbp_lv_location", "tbp_lv_location_simple",'patient_id',
   'anatom_site_general','copyright_license','attribution','image_type']

In [10]:
cb_params = {
    'objective': 'Logloss',
    "random_state": 8640,
    "cat_features": cat_cols,
    "max_depth": 7,
    "l2_leaf_reg": 5,
    "task_type": "GPU",
    "verbose": 0,
}

cb_Model = cb.CatBoostClassifier(**cb_params)

In [11]:
k_folds = 5
trained_cb,test_preds,all_models = train_model(cb_Model,x_train_df,y_train_df,x_test_df,k_folds)

Fold 1: Train pAUC = 0.1883, Validation pAUC = 0.1653
Fold 2: Train pAUC = 0.1887, Validation pAUC = 0.1718
Fold 3: Train pAUC = 0.1867, Validation pAUC = 0.1615
Fold 4: Train pAUC = 0.1907, Validation pAUC = 0.1586
Fold 5: Train pAUC = 0.1860, Validation pAUC = 0.1780

Mean Train pAUC: 0.1881
Mean Validation pAUC: 0.1670


In [12]:
test_outputs = np.zeros(len(x_test_df))

for i in range(k_folds) :
    test_outputs += test_preds[i]
    
test_outputs /= k_folds

df_sample_submission = pd.DataFrame({'isic_id' : test_meta_or['isic_id'],'target' : test_outputs}) 



In [13]:
df_sample_submission

,isic_id,target
0,ISIC_0015657,0.000227
1,ISIC_0015729,0.000021
2,ISIC_0015740,0.000519


In [14]:
df_sample_submission.to_csv('/kaggle/working/submission.csv',index=False)